![Assimila logo](../helpers/logosmall.png "Assimila")
# Assimila Peatlands Demonstrator
# Compare bounding box for 2 time steps

If the drop down menus do not appear restart the notebook by selecting "Kernel" then "Restart" above.

In [3]:
import sys
sys.path.append("../helpers")
import os
from data import Data
from maptools import MapTools
from widgets import Widgets

import matplotlib
%matplotlib notebook
# matplotlib.use("nbagg")
from IPython.display import clear_output

from ipyleaflet import (Map, Marker, basemaps, basemap_to_tiles, 
                        TileLayer, ImageOverlay, Polyline, Polygon, 
                        Rectangle,  GeoJSON, WidgetControl,  DrawControl, 
                        LayerGroup, FullScreenControl, interactive, ScaleControl)

import ipywidgets as widgets
import datetime


def on_button_clicked(b):
    d = Data(out2, keyfile = None)
    
    if operation.value == 'Average of one sub-product':
        m1 = d.color_map_nesw_compare_reduced(product1.value, 
                                              subproduct1.value, 
                                              north.value,
                                              east.value, 
                                              south.value, 
                                              west.value,
                                              date1.value, 
                                              date2.value)
    
    
    if operation.value == 'Subtraction of one sub-product':
        m1 = d.color_map_subtraction(product1.value, 
                                      subproduct1.value, 
                                      north.value,
                                      east.value, 
                                      south.value, 
                                      west.value,
                                      date1.value, 
                                      date2.value)
        
    
    if operation.value == 'Trend analysis for one sub-product':
        m1 = d.trend_analysis(product1.value, 
                              subproduct1.value, 
                              trends.value, 
                              north.value, 
                              east.value, 
                              south.value, 
                              west.value, 
                              date1.value, 
                              date2.value, 
                              date3.value, 
                              date4.value)
        
    if operation.value == 'Identifying change':
        dates = []
        for item in date_carousel.children:
            dates.append(item.value)
        

        
        m = d.color_map_identifying_change(product1.value, 
                                           subproduct1.value, 
                                           north.value, 
                                           east.value,
                                           south.value, 
                                           west.value, 
                                           dates)
    


def on_loc_button_clicked(b):

    if north.value and not south.value and east.value and not west.value:
        m.add_map_point(east.value, north.value)
        m.map.center = (north.value, east.value)
        m.map.zoom = 6
        
    elif not north.value and south.value and not east.value and west.value:
        m.add_map_point(east.value, north.value)
        m.map.center = (south.value, west.value)
        m.map.zoom = 6
        
    elif north.value and east.value and south.value and west.value:
        if north.value == south.value and east.value == west.value:
            m.add_map_point(east.value, north.value)
            m.map.center = (south.value, west.value)
            m.map.zoom = 6
        else:
            m.add_map_rect(north.value, east.value, south.value, west.value)
            m.map.center = (float((north.value+south.value)/2), float((east.value+west.value)/2))
            m.map.zoom = 6

def update_north(x):
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (north, 'value'))

def update_east(x):
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (east, 'value'))

def update_south(x):
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (south, 'value'))

def update_west(x):
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (west, 'value'))    
    

def handle_draw(self, action, geo_json):
    w = Widgets()
    s = geo_json.get('geometry','type')
    t = s.get('type')
    
    if t == 'Point':
        north, east, south, west = MapTools.get_coords_point(geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

    if t == 'Polygon':
        north, east, south, west = MapTools.get_coords_polygon(geo_json)
        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)

def update(*args):
    if product1.value == ' ':
        subproduct1.layout.visibility = 'hidden'

    elif operation.value != 'Trend analysis for one sub-product':
        subproduct1.layout.visibility = 'visible'
        subproduct1.options = w.get_subproduct_list(product1.value)
    
    else:
        subproduct1.layout.visibility = 'visible'
        

    if product2.value == ' ':
        subproduct2.layout.visibility = 'hidden'
    
    else:
        subproduct2.layout.visibility = 'visible'
        subproduct2.options = w.get_subproduct_list(product2.value)
    
    if operation.value == 'Subtraction of one sub-product':
        product2.value = product1.value
        subproduct2.value = subproduct1.value
        product2.disabled = True
        subproduct2.disabled = True
    
    if operation.value == 'Trend analysis for one sub-product':
        subproduct1.options = w.get_subproduct_trend_analysis(product1.value)
    
    
def update_proj(*args):
    new_proj = args[0]['new']
    old_proj = args[0]['old']

    d = Data(out)
    
    if new_proj == 'WGS84' and old_proj == 'BNG':
        lat1, lon1 = d.bng_to_latlon(south.value, west.value, to_latlon=True)
        lat2, lon2 = d.bng_to_latlon(north.value, east.value, to_latlon=True)
        update_north(lat2)
        update_east(lon2)
        update_south(lat1)
        update_west(lon1)
        
        north.description = 'North (lat):'
        east.description = 'East (lon):'
        south.description = 'South (lat):'
        west.description = 'West (lon):'

    if new_proj == 'BNG' and old_proj == 'WGS84':
        nort1, east1 = d.bng_to_latlon(south.value, west.value)
        nort2, east2 = d.bng_to_latlon(north.value, east.value)
        update_north(nort2)
        update_east(east2)
        update_south(nort1)
        update_west(east1)
        
        north.description = 'North (nort):'
        east.description = 'East (east):'
        south.description = 'South (nort):'
        west.description = 'West (east):'
        
    if new_proj == 'Sinusoidal':
        pass

def update_file(*args):
    # TODO: m.map.remove_layer(GeoJSON)
    change = args[0]
    # Get shapefile name and pass into func
    fnames = list(change['owner'].value)
    
    # Write file locally 
    for count, fname in enumerate(fnames):
        with open(fname, 'wb') as outf:
            content = change['owner'].data[count]
            outf.write(content)
    
    # Skip conversion if already .geojson
    if '.geojson' in fname:
        m.add_geojson(fname)
    
    else:
        # convert .shp to .geojson
        Data.shape_to_geojson(fname, 'POLYGON.geojson')
        north, east, south, west = m.add_geojson('POLYGON.geojson')

        update_north(north)
        update_east(east)
        update_south(south)
        update_west(west)
    

def update_operation(*args):
    change = args[0]
    operation = change['new']
    if operation == 'Average of one sub-product':
        product2.disabled = True
        subproduct2.disabled = True
        
        date1.disabled = False
        date2.disabled = False
        date3.disabled = True
        date4.disabled = True
        
        average.disabled = False
        trends.disabled = False
        frequency.disabled = False
        frequency.options = [' ', 'days', 'months', 'years']
    
    if operation == 'Subtraction of one sub-product':
        product2.disabled = True
        subproduct2.disabled = True
        
        date1.disabled = False
        date2.disabled = False
        date3.disabled = True
        date4.disabled = True

        # date_carousel.children.disabled = True
        average.disabled = True
        trends.disabled = True
        frequency.disabled = True
        
        product2.value = product1.value
        subproduct2.value = subproduct1.value
    
    if operation == 'Trend analysis for one sub-product':
        product1.options = [' ', 'MOD13A2', 'MCD43A3', 'era5']
        product2.disabled = True
        subproduct2.disabled = True
        
        date1.disabled = False
        date2.disabled = False
        date3.disabled = False
        date4.disabled = False
        
        average.disabled = True
        trends.disabled = False
        frequency.disabled = True
        
        
        
    if operation == 'Identifying change':
        product2.disabled = True
        
        date1.disabled = True
        date2.disabled = True
        date3.disabled = True
        date4.disabled = True

        average.disabled = True
        trends.disabled = True
        frequency.disabled = False
        
        
def update_change(*args):
    n = args[0]['new']
    dates = []
    for i in range(n):
        dates.append(w.get_date(value=datetime.date(2018,1,1), description=f'Date {i+1}:'))
    date_carousel.children = dates

           
# Map 
m = MapTools(center=(55, -3), zoom=5, width='400px', height='600px')
m.map.add_control(m.dc)
m.map.add_control(ScaleControl(position='bottomleft'))
m.prepare_map()

# Widgets
w = Widgets()

operation = w.operation()

product1 = w.product('Product 1:')
subproduct1 = w.subproduct('Subproduct 1:', 'subproduct1')

product2 = w.product('Product 2:')
subproduct2 = w.subproduct('Subproduct 2:', 'subproduct2')

projection = w.projection()
file_upload = w.upload_file()

average = w.average()
trends = w.trends()
frequency = w.frequency()
date_carousel = w.date_carousel()


#TODO: None doesn't seem to set value 
north = w.get_point(None, 'North (lat): ')
east = w.get_point(None, 'East (lon): ')
south = w.get_point(None, 'South (lat): ')
west = w.get_point(None, 'West (lon): ')
date1 = w.get_date(datetime.date(2018,1,1), 'Start 1: ')
date2 = w.get_date(datetime.date(2018,1,17), 'End 1: ')
date3 = w.get_date(datetime.date(2018,1,1), 'Start 2: ')
date4 = w.get_date(datetime.date(2018,1,17), 'End 2: ')

button = w.set_up_button(on_button_clicked)
button_loc = w.set_up_loc_button(on_loc_button_clicked)

w.display_widget_comparison_reduced(operation, product1, subproduct1, product2, subproduct2, 
                                    projection, date_carousel, north, east,south, west, button_loc, date1, 
                                    date2, date3, date4, file_upload, button, m.map, average, trends, frequency)

# w.gridspec_display(operation, product1, subproduct1, product2, subproduct2, 
#                      projection, north, east, south, west, button_loc, date1,  
#                      date2, date3, date4, file_upload, button, m.map, average, trends, change)


# Events
label = widgets.Label()
m.mouse_interaction(label)
        
update()

product1.observe(update)
product2.observe(update)
subproduct1.observe(update)

projection.observe(update_proj, names='value')
file_upload.observe(update_file, names='data')
operation.observe(update_operation, names='value')
frequency.observe(update_change, names='value')

out = w.display_output()

x = m.dc.on_draw(handle_draw) 

#############
# Plot Area #
#############

output_layout = widgets.Layout(width='100%', height='550px', border='5px solid grey')
out2 = w.display_output()
out2.layout = output_layout


# from ipywidgets import GridspecLayout, Layout

# grid = GridspecLayout(10,10)
# grid.layout = widgets.Layout(width='100%', height='500px')

# with out2:
#     display(grid)

Label(value='')

Output()

Output()

In [3]:
from ipywidgets import GridspecLayout, Layout

# grid = GridspecLayout(20,20)
# grid.layout.height = '2000px'

# grid[:1, :8] = widgets.Button(button_style='danger')
# grid[:9, :8] = widgets.Button(button_style='danger')

# grid[:2, 9:11] = widgets.Button(button_style='danger')
# grid[2, 9:12] = widgets.Button(button_style='danger')

# grid[0, 12:16] = widgets.Button(button_style='danger')
# grid[2:3, 12:16] = widgets.Button(button_style='danger')
# grid[2, 12:16] = widgets.Button(button_style='danger')
# grid[3, 12:16] =widgets.Button(button_style='danger')

# grid[:3, 16:] = widgets.Button(button_style='danger')

# grid[3:5, 9:13] = widgets.Button(button_style='danger')
# grid[6:8, 9:13] = widgets.Button(button_style='danger')
# grid[3:5, 13:16] = widgets.Button(button_style='danger')
# grid[4:6, 13:18] = widgets.Button(button_style='danger')

# grid[7:8, 9:11] = widgets.Button(button_style='danger')
# grid[8:9, 9:11] = widgets.Button(button_style='danger')
# grid[7:8, 13:18] = widgets.Button(button_style='danger')
# grid[8:9, 13:17] = widgets.Button(button_style='danger')

# grid[9:11, :5] = widgets.Button(button_style='danger')
# grid[9:11, 6:10] = widgets.Button(button_style='danger')
# grid[9:11, 13:20] = widgets.Button(button_style='danger')

# display(grid)

grid = GridspecLayout(40,20)
grid.layout.height = '1600px'
m.map.height = 'auto'
grid[0, :8] = operation
grid[1:9, :8] = m.map

grid[:2, 10:11] = file_upload
grid[2, 10:11] = button_loc

grid[0, 11:14] = widgets.BoundedFloatText(description='n', layout=Layout(width='200px'))
grid[1, 11:14] = widgets.BoundedFloatText(description='e', layout=Layout(width='200px'))
grid[2, 11:14] = widgets.BoundedFloatText(description='s', layout=Layout(width='200px'))
grid[3, 11:14] = widgets.BoundedFloatText(description='w', layout=Layout(width='200px'))

grid[:3, 14:] = projection

grid[4, 10:13] = widgets.Button(description='product1')
grid[5, 10:13] = widgets.Button(description='date1')
grid[4, 13:18] = widgets.Button(description='subproduct1')
grid[5, 13:18] = widgets.Button(description='date2')

grid[7, 10:11] = widgets.Button(description='product2')
grid[8, 10:11] = widgets.Button(description='date3')
grid[7, 13:18] = widgets.Button(description='subproduct2')
grid[8, 13:17] = widgets.Button(description='date4')

grid[12, 11] = average
grid[13, 11] = trends
grid[14, 11] = change

display(grid)


NameError: name 'm' is not defined

## Available MODIS products:

  - __MOD11A1 - Land surface temperature__
  - __MOD13A2 - Vegetation indices__
  - __MCD43A3 - Albedo__


## MOD11A1 - Land surface temperature
Daily, 1km resolution
  - Day and night __land surface temperature__ (LST_Day_1km, LST_night_1km)

Also:
 - __Quality control flags__ (day and night).
   - These need interperating before using.
 - Day and night __view time__.
 - Day and night __view angle__.
 - __Clear sky coverage__ (day/night).
 - __Emissivity__ (band 31 and 32)


## MOD13A2 - Vegetation indices
Every 16 days, 1 km resolution
  - Vegetation indices: __NDVI__, __EVI__

Also:
  - Vegetation index __quality indicators__
  - __Reflectance__ for red, NIR, blue and MIR bands
  - Viewing and sun angles: __view zenith__, __sun zenith__ and __relative azimuth__
  - __Pixel reliability__

## MCD43A3 - Albedo
Daily, 500 m resolution
  - __White sky albedo__ for each band
  - __Black sky albedo__ for each band
  - __Quality indicators__
    - need interpretation